In [1]:
import io
from pydub import AudioSegment
import speech_recognition as sr
import whisper
import queue
import tempfile
import os
import threading
import click
import torch
import numpy as np


In [2]:
def record_audio(audio_queue, energy, pause, dynamic_energy, save_file, temp_dir):
    #load the speech recognizer and set the initial energy threshold and pause threshold
    r = sr.Recognizer()
    r.energy_threshold = energy
    r.pause_threshold = pause
    r.dynamic_energy_threshold = dynamic_energy

    with sr.Microphone(sample_rate=16000) as source:
        print("Say something!")
        i = 0
        while True:
            #get and save audio to wav file
            audio = r.listen(source)
            if save_file:
                data = io.BytesIO(audio.get_wav_data())
                audio_clip = AudioSegment.from_file(data)
                filename = os.path.join(temp_dir, f"temp{i}.wav")
                audio_clip.export(filename, format="wav")
                audio_data = filename
            else:
                torch_audio = torch.from_numpy(np.frombuffer(audio.get_raw_data(), np.int16).flatten().astype(np.float32) / 32768.0)
                audio_data = torch_audio

            audio_queue.put_nowait(audio_data)
            i += 1


def transcribe_forever(audio_queue, result_queue, audio_model, english, verbose, save_file):
    while True:
        audio_data = audio_queue.get()
        if english:
            result = audio_model.transcribe(audio_data,language='english')
        else:
            result = audio_model.transcribe(audio_data)

        if not verbose:
            predicted_text = result["text"]
            result_queue.put_nowait("You said: " + predicted_text)
        else:
            result_queue.put_nowait(result)

        if save_file:
            os.remove(audio_data)


In [3]:
model="base"
english=True
verbose=False
energy=100
pause=0.8
dynamic_energy=False
save_file=False

temp_dir = tempfile.mkdtemp() if save_file else None
#there are no english models for large
if model != "large" and english:
    model = model + ".en"

In [4]:
audio_model = whisper.load_model(model).to("cuda")

In [5]:
audio_queue = queue.Queue()
result_queue = queue.Queue()
record_thread = threading.Thread(target=record_audio,
                    args=(audio_queue, energy, pause, dynamic_energy, save_file, temp_dir))
record_thread.start()
transcribe_thread = threading.Thread(target=transcribe_forever,
                    args=(audio_queue, result_queue, audio_model, english, verbose, save_file))
transcribe_thread.start()

Say something!


In [12]:
while True:
    print(result_queue.get())

You said:  Apparently not.
You said: 
You said:  Pewter window to the left


KeyboardInterrupt: 